# Exploración de Datos - BIOMASS

Este notebook contiene la exploración inicial de los datos del proyecto BIOMASS.

## 1. Importar librerías necesarias

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os

# Configuración de visualización
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

# Configuración de pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [ ]:
# Definir ruta base de datos
base_path = "../data/"

## 2. Cargar datos

In [ ]:
# Cargar datasets
train_df = pd.read_csv(os.path.join(base_path, 'train.csv'))
test_df = pd.read_csv(os.path.join(base_path, 'test.csv'))

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")

## 3. Función para cargar imágenes

In [ ]:
def load_image(image_path, img_base_path=None):
    """
    Carga y retorna una imagen desde el directorio de datos.
    
    Parameters:
    -----------
    image_path : str
        Ruta relativa de la imagen (ej: 'train/ID1011485656.jpg')
    img_base_path : str, optional
        Ruta base donde están los datos. Si no se especifica, usa la variable global base_path
    
    Returns:
    --------
    PIL.Image
        Imagen cargada
    """
    if img_base_path is None:
        img_base_path = base_path
    
    full_path = os.path.join(img_base_path, image_path)
    
    if not os.path.exists(full_path):
        raise FileNotFoundError(f"No se encontró la imagen en: {full_path}")
    
    img = Image.open(full_path)
    return img

# Función auxiliar para mostrar imágenes
def show_image(image_path, title=None, figsize=(8, 6)):
    """
    Muestra una imagen con matplotlib.
    
    Parameters:
    -----------
    image_path : str
        Ruta relativa de la imagen
    title : str, optional
        Título para la imagen
    figsize : tuple
        Tamaño de la figura
    """
    img = load_image(image_path)
    plt.figure(figsize=figsize)
    plt.imshow(img)
    plt.axis('off')
    if title:
        plt.title(title, fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

## 4. Exploración Básica de Datos (EDA)

### 4.1 Primeras filas y estructura del dataset

In [ ]:
# Primeras filas del dataset de entrenamiento
train_df.head(10)

In [ ]:
# Información del dataset
train_df.info()

In [ ]:
# Estadísticas descriptivas
train_df.describe()

### 4.2 Valores nulos y duplicados

In [ ]:
# Valores nulos
print("Valores nulos por columna:")
print(train_df.isnull().sum())
print(f"\nTotal de valores nulos: {train_df.isnull().sum().sum()}")

# Duplicados
print(f"\nFilas duplicadas: {train_df.duplicated().sum()}")

### 4.3 Análisis de variables categóricas

In [ ]:
# Estados
print("Distribución de Estados:")
print(train_df['State'].value_counts())
print(f"\nNúmero de estados únicos: {train_df['State'].nunique()}")

# Especies
print("\n" + "="*50)
print("Distribución de Especies:")
print(train_df['Species'].value_counts())
print(f"\nNúmero de especies únicas: {train_df['Species'].nunique()}")

# Target names
print("\n" + "="*50)
print("Distribución de target_name:")
print(train_df['target_name'].value_counts())
print(f"\nNúmero de targets únicos: {train_df['target_name'].nunique()}")

In [ ]:
# Número de imágenes únicas
print(f"Número de imágenes únicas: {train_df['image_path'].nunique()}")
print(f"Total de filas: {len(train_df)}")
print(f"Promedio de targets por imagen: {len(train_df) / train_df['image_path'].nunique():.2f}")

### 4.4 Análisis de la variable objetivo (target)

In [ ]:
# Distribución del target
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histograma
axes[0].hist(train_df['target'], bins=50, edgecolor='black')
axes[0].set_title('Distribución del Target', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Target (g)')
axes[0].set_ylabel('Frecuencia')
axes[0].grid(alpha=0.3)

# Boxplot
axes[1].boxplot(train_df['target'])
axes[1].set_title('Boxplot del Target', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Target (g)')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Estadísticas
print(f"Media: {train_df['target'].mean():.2f} g")
print(f"Mediana: {train_df['target'].median():.2f} g")
print(f"Desviación estándar: {train_df['target'].std():.2f} g")
print(f"Mínimo: {train_df['target'].min():.2f} g")
print(f"Máximo: {train_df['target'].max():.2f} g")

In [ ]:
# Target por tipo
fig, axes = plt.subplots(2, 1, figsize=(12, 10))

# Boxplot por target_name
train_df.boxplot(column='target', by='target_name', ax=axes[0])
axes[0].set_title('Distribución del Target por Tipo de Medición', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Tipo de Target')
axes[0].set_ylabel('Target (g)')
plt.sca(axes[0])
plt.xticks(rotation=45, ha='right')

# Media por target_name
target_means = train_df.groupby('target_name')['target'].mean().sort_values()
axes[1].barh(target_means.index, target_means.values)
axes[1].set_title('Media del Target por Tipo de Medición', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Media del Target (g)')
axes[1].grid(alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

### 4.5 Análisis de características numéricas

In [ ]:
# Distribución de Pre_GSHH_NDVI
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].hist(train_df['Pre_GSHH_NDVI'], bins=30, edgecolor='black')
axes[0].set_title('Distribución de Pre_GSHH_NDVI', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Pre_GSHH_NDVI')
axes[0].set_ylabel('Frecuencia')
axes[0].grid(alpha=0.3)

axes[1].hist(train_df['Height_Ave_cm'], bins=30, edgecolor='black')
axes[1].set_title('Distribución de Altura Promedio', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Height_Ave_cm')
axes[1].set_ylabel('Frecuencia')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

### 4.6 Correlaciones

In [ ]:
# Matriz de correlación
numeric_cols = ['Pre_GSHH_NDVI', 'Height_Ave_cm', 'target']
corr_matrix = train_df[numeric_cols].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, fmt='.3f', cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Matriz de Correlación', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Scatter plots
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].scatter(train_df['Pre_GSHH_NDVI'], train_df['target'], alpha=0.5)
axes[0].set_title('Target vs Pre_GSHH_NDVI', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Pre_GSHH_NDVI')
axes[0].set_ylabel('Target (g)')
axes[0].grid(alpha=0.3)

axes[1].scatter(train_df['Height_Ave_cm'], train_df['target'], alpha=0.5)
axes[1].set_title('Target vs Altura Promedio', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Height_Ave_cm')
axes[1].set_ylabel('Target (g)')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

### 4.7 Análisis por Estado y Especie

In [ ]:
# Target promedio por Estado
state_target = train_df.groupby('State')['target'].mean().sort_values()

plt.figure(figsize=(10, 5))
plt.barh(state_target.index, state_target.values)
plt.title('Target Promedio por Estado', fontsize=14, fontweight='bold')
plt.xlabel('Target Promedio (g)')
plt.ylabel('Estado')
plt.grid(alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

In [ ]:
# Target promedio por Especie (top 15)
species_target = train_df.groupby('Species')['target'].mean().sort_values(ascending=False).head(15)

plt.figure(figsize=(12, 6))
plt.barh(species_target.index, species_target.values)
plt.title('Top 15 Especies por Target Promedio', fontsize=14, fontweight='bold')
plt.xlabel('Target Promedio (g)')
plt.ylabel('Especie')
plt.grid(alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

### 4.8 Ejemplo de carga de imagen

In [ ]:
# Mostrar una imagen de ejemplo
sample_image_path = train_df['image_path'].iloc[0]
sample_info = train_df[train_df['image_path'] == sample_image_path].iloc[0]

print(f"Información de la muestra:")
print(f"Sample ID: {sample_info['sample_id']}")
print(f"Species: {sample_info['Species']}")
print(f"State: {sample_info['State']}")
print(f"Height: {sample_info['Height_Ave_cm']} cm")
print(f"NDVI: {sample_info['Pre_GSHH_NDVI']}")
print(f"Target: {sample_info['target']} g ({sample_info['target_name']})")

show_image(sample_image_path, title=f"Ejemplo: {sample_info['Species']} - {sample_info['State']}")